In [1]:
import pandas as pd
import os
import numpy as np
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense,Dropout,Flatten,Activation
from keras.layers.convolutional import Conv2D,MaxPooling2D,ZeroPadding2D,AveragePooling2D
from keras.utils import np_utils
import skimage
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
import cv2

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.color import rgb2gray

In [4]:
os.getcwd()

'C:\\Users\\AnKajVi\\Documents'

In [5]:
os.chdir('./chest-xray-pneumonia/chest_xray/train')

In [6]:
os.getcwd()

'C:\\Users\\AnKajVi\\Documents\\chest-xray-pneumonia\\chest_xray\\train'

In [7]:
for i in os.listdir('./'):
    print(i)

NORMAL
PNEUMONIA


In [8]:
for i in os.listdir('./'):
    count=0
    for j in os.listdir('./'+i):
        count+=1
    print(i +':')
    print(count)

NORMAL:
1341
PNEUMONIA:
3875


In [9]:
os.getcwd()

'C:\\Users\\AnKajVi\\Documents\\chest-xray-pneumonia\\chest_xray\\train'

In [10]:
def get_data(end):
    x=[]
    y=[]
    label=0
    print("Reading_Files")
    file=0
    for i in os.listdir('./'):
        if(i=='PNEUMONIA'):
            print("----PNEUMONIA------")
            #label=1
            cnt=0
            for j in os.listdir('./'+'PNEUMONIA'):
                img_file = mpimg.imread('./'+i+'/'+j)
                img_file=rgb2gray(img_file)
                img_file = skimage.transform.resize(img_file, (100,100,1))
                img_arr = np.asarray(img_file)
                x.append(img_arr)
                y.append(1)
                cnt+=1
                if cnt>=end: 
                    break
        else:
            print("----NORMAL-----")
            cnt=0
            #label=0
            for j in os.listdir('./'+i):
                img_file = mpimg.imread('./'+i+'/'+j)
                img_file=rgb2gray(img_file)
                img_file = skimage.transform.resize(img_file, (100,100,1))
                img_arr = np.asarray(img_file)
                x.append(img_arr)
                y.append(0)
                cnt+=1
                if cnt>=end:
                    break;
    x = np.asarray(x)
    y = np.asarray(y)
    print("----------------READING COMPLETE--------------")
    return x,y

In [11]:
x,y = get_data(1000)
y=np_utils.to_categorical(y,num_classes=2)
print(y)
x,y= shuffle(x,y,random_state=13)

Reading_Files
----NORMAL-----
----PNEUMONIA------
----------------READING COMPLETE--------------
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [12]:
print(y.shape)

(2000, 2)


In [13]:
import keras
from keras.layers import Input,Flatten,Dense,Reshape,Dropout
from keras.layers import BatchNormalization,Activation,ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.layers import Add
from keras.optimizers import Adam

In [ ]:
def get_model():
    input_layer = Input(shape=(100,100,1))
    hid = Conv2D(32,(2,2),padding='same', activation='relu')(input_layer)
    hid = MaxPooling2D(pool_size=(3,3))(hid)
    hid = Conv2D(64,(2,2),padding='same', activation='relu')(hid)
    hid = MaxPooling2D(pool_size=(3,3))(hid)
    hid2 = hid
    hid = Conv2D(64,(2,2),padding='same', activation='relu')(hid)
    hid = Add()([hid2,hid])
    
    hid = MaxPooling2D(pool_size=(3,3))(hid)

    hid = Flatten()(hid)
    
    hid = Dense(32)(hid)
    out = Dense(2, activation='softmax')(hid)
    
    model = Model(inputs= input_layer, outputs= out)
    model.summary()
    
    return model

In [ ]:
model = get_model()

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
filepath = 'C:\\Users\\AnKajVi\\Documents\\Xray2\\nnweight-weights.{epoch:02d}-{loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(x,y,epochs=2,batch_size=128,verbose=1,callbacks=callbacks_list)

In [ ]:
def get_test_data(end):
    x=[]
    y=[]
    label=0
    print("Reading_Files")
    file=0
    for i in os.listdir('../test'):
        if(i=='PNEUMONIA'):
            cnt=0
            print("----PNEUMONIA------")
            for j in os.listdir('../test/'+'PNEUMONIA'):
                img_file = mpimg.imread('../test/'+i+'/'+j)
                img_file=rgb2gray(img_file)
                img_file = skimage.transform.resize(img_file, (100,100,1))
                img_arr = np.asarray(img_file)
                x.append(img_arr)
                y.append(1)
                cnt+=1
                if cnt>=end:
                    break
        else:
            cnt=0
            print("----NORMAL-----")
            for j in os.listdir('./'+i):
                img_file = mpimg.imread('../test/'+i+'/'+j)
                img_file=rgb2gray(img_file)
                img_file = skimage.transform.resize(img_file, (100,100,1))
                img_arr = np.asarray(img_file)
                x.append(img_arr)
                y.append(0)
                cnt+=1
                if cnt>=end:
                    break
    x = np.asarray(x)
    y = np.asarray(y)
    print("----------------READING COMPLETE--------------")
    return x,y

In [ ]:
x_test,y_test = get_test_data(150)

In [ ]:
y_test = np_utils.to_categorical(y_test,num_classes=2)
print(y_test.shape)

In [42]:
_, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [43]:
print('Test: %.3f' % (test_acc))

Test: 0.843
